<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/contrast/iclaim_items.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
iclaim = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd150000.csv')

In [6]:
a = iclaim[(iclaim['order_item']!='[]') & (iclaim['billing']!='[]')]
a.columns

Index(['transaction_uid', 'transaction_no', 'hospital_code', 'hospital_name',
       'visit_date', 'dx_free_text', 'claim_status', 'coma_score',
       'discharge_at', 'approve_at', 'icd9', 'vital_sign', 'billing',
       'order_item', 'icd_10', 'policy_detail'],
      dtype='object')

In [7]:
a.sort_values(by=['visit_date'], inplace=True, ascending=True)

<ipython-input-7-e523b473d8cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a.sort_values(by=['visit_date'], inplace=True, ascending=True)


In [10]:
a.to_csv('/content/drive/MyDrive/AIEngineer/contrast/iclaimdata_3months.csv', index=False)

In [12]:
a['order_item'].iloc[0]

"[{'initial': '300', 'discount': '0', 'net_amount': '300.00', 'item_amount': '1', 'item_id': 'DRES', 'item_name': 'DRESSING  กลาง[OPD][', 'local_billing_code': 'DRES', 'local_billing_name': 'Packaged Medical Charges  ค่าบริการชุดเหมาจ่าย ', 'simb_billing_code': '1.1.14(1)'}, {'initial': '230', 'discount': '0', 'net_amount': '230.00', 'item_amount': '1', 'item_id': 'SUP1', 'item_name': 'ASKINA TRANSORBENT 1', 'local_billing_code': 'SUP1', 'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1', 'simb_billing_code': '1.1.2(1)'}]"

In [13]:
df = a.copy()

In [15]:
a['icd_10'].iloc[0]

"[{'code': 'Z48.0', 'dx_name': 'Attention to surgical dressings and sutures', 'dx_type': 'PP'}, {'code': 'V2999', 'dx_name': 'Motorcyclist [any] injured in unspecified traffic accident: during unspec activity', 'dx_type': 'EC'}]"

In [20]:
a.head(2)

,transaction_uid,transaction_no,hospital_code,hospital_name,visit_date,dx_free_text,claim_status,coma_score,discharge_at,approve_at,icd9,vital_sign,billing,order_item,icd_10,policy_detail
147266,771dcc21-d407-4d3a-a676-2b53efa8a3be,3cc5862a-adbb-4485-8910-ea13fee1e3d3,11558,โรงพยาบาลซีจีเอช พหลโยธิน,2024-07-15 16:30,mc ล้ม,Submit,15.0,2024-08-09 18:12,2024-08-09 18:12,[],"[{'diastolic_bp': '97', 'systolic_bp': '156', ...","[{'simb_code': '1.1.14(1)', 'simb_description'...","[{'initial': '300', 'discount': '0', 'net_amou...","[{'code': 'Z48.0', 'dx_name': 'Attention to su...","{'insurance': 'aia', 'policy_no': 'P253132440'..."
145720,49c60f16-5639-4ee2-8c9c-200551bb2009,ea14c9c0-39f5-486e-b9ff-d8ea6e35a211,11787,โรงพยาบาลเกษมราษฎร์อินเตอร์เนชั่นแนล รัตนาธิเบศร์,2024-07-20 19:10,ิวหนังและเนื้อเยื่ออักเสบจากการแพ้,Approve,NaN,2024-08-10 16:02,2024-08-10 16:02,[],"[{'diastolic_bp': '79.00', 'systolic_bp': '120...","[{'simb_code': '1.2.1(1)', 'simb_description':...","[{'initial': '960', 'discount': '0', 'net_amou...","[{'code': 'W010', 'dx_name': 'Fall on same lev...","{'insurance': 'aia', 'policy_no': 'P255787152'..."


In [23]:
import ast

# Expanding icd_10 and order_item based on dx_type == 'PP'
expanded_rows = []
for _, row in df.iterrows():
    icd_10_items = ast.literal_eval(row["icd_10"])  # Convert string to list of dicts
    order_items = ast.literal_eval(row["order_item"])  # Convert order_item string to list

    for icd in icd_10_items:
        if icd["dx_type"] == "PP":
            for item in order_items:
                combined_entry = {
                    "icd10_PP": icd["code"],
                    "initial": item["initial"],
                    "discount": item["discount"],
                    "net_amount": item["net_amount"],
                    "item_amount": item["item_amount"],
                    "item_id": item["item_id"],
                    "item_name": item["item_name"],
                    "local_billing_code": item["local_billing_code"],
                    "local_billing_name": item["local_billing_name"],
                    "simb_billing_code": item["simb_billing_code"]
                }
                expanded_rows.append(combined_entry)

item = pd.DataFrame(expanded_rows)

In [24]:
item

,icd10_PP,initial,discount,net_amount,item_amount,item_id,item_name,local_billing_code,local_billing_name,simb_billing_code
0,Z48.0,300,0,300.00,1,DRES,DRESSING กลาง[OPD][,DRES,Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,1.1.14(1)
1,Z48.0,230,0,230.00,1,SUP1,ASKINA TRANSORBENT 1,SUP1,Medical Supplies 1 เวชภัณฑ์1,1.1.2(1)
2,S32600,960,0,960.00,1,DF1,DF แพทย์ตรวจรักษา (Doctor Fee),310,First Outpatient care before admission ค่าแพท...,1.2.1(1)
3,S32600,290,29,261.00,1,0414C10E0005O01,E--lo--met cream # (LASA),50,Outpatient Medication ค่ายาผู้ป่วยนอก,1.1.1(3)
4,S32600,185,0,185.00,1,NSE,ค่าบริการพยาบาล ผู้ป่วยนอก (Out Patient Servic...,240,Nursing and Midwifery Charge ค่าบริการพยาบาล,1.1.12
...,...,...,...,...,...,...,...,...,...,...
354965,S60,30,0,30.00,1,NS800001.D,ค่าบริการทางการแพทย์อื่น ๆ นิติเวช,1.1.14(2),Other Medical Service Charge,1.1.14(2)
354966,S60,200,0,200.00,1,F2210001,ค่าแพทย์ตรวจทั่วไป,1.1.14(2),Other Medical Service Charge,1.1.14(2)
354967,S60,400,0,400.00,1,F2210001.D,ค่าแพทย์ตรวจทั่วไป,1.2.1(1),First Outpatient care before admission ค่าแพท...,1.2.1(1)
354968,S60,335,0,335.00,1,GE130101,CHEST,1.1.5(1),Standard Diagnostic ค่าตรวจรังสีปกติ รังสี...,1.1.5(1)


In [27]:
item.to_csv('/content/drive/MyDrive/AIEngineer/contrast/iclaim_item.csv', index=False)

In [28]:
w = pd.read_excel('/content/drive/MyDrive/AIEngineer/contrast/Billing_Mapping_รพ.วานรนิวาส.xlsx')
s = pd.read_excel('/content/drive/MyDrive/AIEngineer/contrast/Billing_Mapping_Samutsakorn.xlsx')

In [49]:
item['local_billing_name'].unique()

array(['Packaged Medical Charges  ค่าบริการชุดเหมาจ่าย ',
       'Medical Supplies 1   เวชภัณฑ์1',
       'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก',
       'Outpatient Medication   ค่ายาผู้ป่วยนอก',
       'Nursing and Midwifery Charge   ค่าบริการพยาบาล',
       'Other Medical Service Charge ค่าบริการทางการแพทย์',
       'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด',
       'General Medical Equipment  ค่าอุปกรณ์นอกห้องผ่าตัด',
       'Special Diagnositics Physician Service  ค่าตรวจวินิจฉัยสาขาเฉพาะ ค่าวิเคราะห์ภาพการแพทย์ คลื่นหัวใจ และอื่นๆ ',
       'Physician Evaluation and Management Services',
       'ค่าสารอาหารทางเส้นเลือด', 'Physical  Therapy   ค่ากายภาพบำบัด',
       'Laboratory Investigation and Pathology  ค่าตรวจทางเทคนิค พยาธิวิทยา',
       'Inpatient Medication   ค่ายาผู้ป่วยใน',
       'Standard Diagnostic    ค่าตรวจรังสีปกติ  รังสีเอ็กซ์',
       'Special Diagnostic   ค่าตรวจวินิจฉัยโดยวิธีพิเศษอื่นๆ',
       'Special Diagnostic Imagin

In [50]:
item[item['icd10_PP']=='K297']['simb_billing_code'].unique()

array(['1.1.12', '1.1.14(2)', '1.1.2(1)', '1.1.4', '1.1.7(1)', '1.2.1(1)',
       '1.1.1(1)', '1.1.1(3)', '1.1.14(1)', '1.2.1', '1.2.1(4)', '2.6(2)',
       '1.1.1(4)', '1.2.1(11)', '1.1.5(2)', '1.2.2', '1.1.5(1)'],
      dtype=object)

In [48]:
item[(item['simb_billing_code']=='1.1.4') & (item['item_name'].str.contains('Glucose'))]['item_name']

,icd10_PP,initial,discount,net_amount,item_amount,item_id,item_name,local_billing_code,local_billing_name,simb_billing_code
4275,K297,350,0,350.00,1,N0002,Glucose (POCT)-Nurse.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
4490,I10,180.6,0,180.60,1,15582,POCT Glucose(Fingertip),7,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
4620,E119,210,0,210.00,1,C1001,Glucose.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
4756,R55,200,0,200.00,1,N0002,Glucose (POCT)-Nurse.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
5883,E119,350,0,350.00,1,C1001,Glucose.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
...,...,...,...,...,...,...,...,...,...,...
335325,E109,200,0,200.00,1,C1001,Glucose.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
338657,E119,250,0,250.00,1,C1001,Glucose.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
342868,E118,350,0,350.00,1,C1001,Glucose.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4
343226,I100,350,0,350.00,1,N0002,Glucose (POCT)-Nurse.,12,Laboratory Investigation and Pathology ค่าตรว...,1.1.4


In [54]:
s['AIA SIMB Billing Code\n(SIMB and Payor Billing Code)'].value_counts()

,count
AIA SIMB Billing Code (SIMB and Payor Billing Code),
1.2.2,2143
1.1.2(1),1881
1.1.4,1463
2.6,489
1.1.5(2),404
1.1.5(1),218
1.1.2(3),183
1.1.9(4),163
1.1.3,151


In [34]:
w[w['AIA SIMB Billing Code\n(SIMB and Payor Billing Code)']=='1.1.4'].head(2)

,nhso_adp_type_id,codehos,nhso_adp_code,billcode,nameitemhos,unitname,price,nhso_adp_type_name,nhso_adp_code_name,codebillhos,codebillhosname,??????????? opd,??????????? opd_1,???????????. ipd,???????????. ipd_1,AIA SIMB Billing Code\n(SIMB and Payor Billing Code),AIA SIMB Description
0,14.0,3000001,22106,22106,Blood group ABO Slide methodx,test,30.0,14-ค่าบริการโลหิต และส่วนประกอบโลหิต,ABO Cell grouping - Slide method (ในกรณีตรวจหม...,5,ค่าบริการโลหิตและส่วนประกอบของโลหิต,6,บริการโลหิตและส่วนประกอบของโลหิต,6,ค่าบริการโลหิตและส่วนประกอบของโลหิต,1.1.4,Laboratory Investigation and Pathology ค่าตรว...
1,14.0,3000002,22105,22105,Blood group ABO Tube method,test,100.0,14-ค่าบริการโลหิต และส่วนประกอบโลหิต,Blood group (ABO- Cell and serum grouping) - T...,5,ค่าบริการโลหิตและส่วนประกอบของโลหิต,6,บริการโลหิตและส่วนประกอบของโลหิต,6,ค่าบริการโลหิตและส่วนประกอบของโลหิต,1.1.4,Laboratory Investigation and Pathology ค่าตรว...


In [30]:
s.head()

,nhso_adp_type_id,codehos,nhso_adp_code,billcode,incomename,name,unit,income,simb_2005,AIA SIMB Billing Code\n(SIMB and Payor Billing Code),AIA SIMB Description
0,2.0,3151951,204,204,NaN,ชุดสายวัดความดันในหลอดเลือดดำ(CVP) (0204),NaN,2,1.1.2(1),1.1.2(1),Medical Supplies 1 เวชภัณฑ์1
1,11.0,3050004,201,201,NaN,ถุงบรรจุศพ,ถุง,16,2.6,2.4,Mortuary Services ค่าบริการเก็บรักษาศพ
2,11.0,3050005,205,205,NaN,Blade surgical stainless(feather),ชิ้น,5,1.1.2(1),1.1.2(1),Medical Supplies 1 เวชภัณฑ์1
3,11.0,3050006,206,206,NaN,ตลับใส่เสมหะ(sputum),ตลับ,5,1.1.2(1),1.1.2(1),Medical Supplies 1 เวชภัณฑ์1
4,11.0,3050007,503,503,NaN,Blood Set (**)(มีที่ห้องยานอก),อัน,5,1.1.2(1),1.1.2(1),Medical Supplies 1 เวชภัณฑ์1


## map with item iclaim  `item_name`



